# **Final Project Task 1 - Census Data Preprocess**

Requirements

- Target variable specification:
    - The target variable for this project is hours-per-week. 
    - Ensure all preprocessing steps are designed to support regression analysis on this target variable.
- Encode data  **3p**
- Handle missing values if any **1p**
- Correct errors, inconsistencies, remove duplicates if any **1p**
- Outlier detection and treatment if any **1p**
- Normalization / Standardization if necesarry **1p**
- Feature engineering **3p**
- Train test split, save it.
- Others?


Deliverable:

- Notebook code with no errors.
- Preprocessed data as csv.

Sunt importate bibliotecile necesare pentru procesul de preprocesare a datelor.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



In [2]:

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


1. Target variable specification

Variabila țintă a acestui proiect este hours-per-week, care reprezintă numărul de ore lucrate pe săptămână de fiecare individ.
Scopul este realizarea unei analize de regresie, motiv pentru care hours-per-week este tratată ca variabilă dependentă numerică, iar toate etapele de preprocesare sunt realizate astfel încât să susțină corect acest tip de analiză.

In [3]:
target = "hours-per-week"

2. Load data

Datele au fost încărcate din Adult Census Dataset (UCI), iar valorile lipsă marcate prin „?” au fost tratate ca valori NaN.

In [4]:
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "income"
]

data = pd.read_csv(data_url, header=None, names=columns, na_values=" ?", skipinitialspace=True)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


3. Handle missing values

Valorile lipsă au fost tratate astfel:

pentru variabilele numerice s-a utilizat mediana;

pentru variabilele categorice s-a utilizat cea mai frecventă valoare (moda).

Această abordare permite păstrarea tuturor observațiilor.

In [5]:
num_cols = data.select_dtypes(exclude="object").columns
cat_cols = data.select_dtypes(include=["object", "string"]).columns

num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

data[num_cols] = num_imputer.fit_transform(data[num_cols])
data[cat_cols] = cat_imputer.fit_transform(data[cat_cols])


4. Correct errors, inconsistencies & remove duplicates

Au fost eliminate observațiile duplicate și au fost corectate eventualele inconsistențe de tip spații inutile din variabilele categorice.


In [6]:
data = data.drop_duplicates()

for col in cat_cols:
    data[col] = data[col].str.strip()


5. Outlier detection and treatment

Pentru variabilele numerice a fost aplicată metoda IQR pentru detectarea valorilor extreme.
Outlierii au fost tratați prin limitare (capping), fără eliminarea observațiilor.


In [7]:
def cap_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.clip(df[col], lower, upper)

for col in num_cols:
    if col != target:
        cap_outliers_iqr(data, col)


6. Feature engineering

Au fost aplicate următoarele tehnici de feature engineering:

eliminarea variabilei education, deoarece informația este deja conținută în education-num;

crearea unei variabile binare care indică existența câștigului sau pierderii de capital;

aplicarea transformării logaritmice pentru variabilele capital-gain și capital-loss.


In [8]:
data = data.drop(columns=["education"])

data["has_capital"] = (
    (data["capital-gain"] > 0) | (data["capital-loss"] > 0)
).astype(int)

data["capital-gain"] = np.log1p(data["capital-gain"])
data["capital-loss"] = np.log1p(data["capital-loss"])


7. Encode data

Variabilele categorice au fost transformate în variabile numerice folosind One-Hot Encoding.

Pentru evitarea multicoliniarității, a fost eliminată prima categorie din fiecare variabilă.



In [9]:
X = data.drop(columns=[target])
y = data[target]

X = pd.get_dummies(X, drop_first=True)


8. Normalization / Standardization

Variabilele numerice au fost standardizate folosind StandardScaler pentru a avea aceeași scară de variație.

Variabila țintă nu a fost standardizată.


In [10]:
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X[X.columns])


9. Train–test split + save

Setul de date a fost împărțit în set de antrenare (80%) și set de testare (20%).

Datele rezultate au fost salvate în format CSV.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.to_csv("X_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_test.to_csv("y_test.csv", index=False)


10. Toate etapele au fost realizate folosind un random_state fix pentru reproductibilitate, iar setul de date rezultat este pregătit pentru etapa de modelare.